## <span style='color:#ff5f27'> 📝 Imports

In [1]:
# !pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [3]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

,user_id,gender,age,country
0,NE254R,Male,74,U.S. Outlying Islands
1,OK667U,Other,72,Zimbabwe
2,TG490T,Male,40,Switzerland
3,MS370Y,Male,27,Sri Lanka
4,FP113J,Female,79,U.S. Virgin Islands


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [4]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

,video_id,category,views,likes,video_length,upload_date
0,1GH88Q,Travel,379726,232338,42,2024-05-08
1,9HG61H,Education,102651,48846,142,2023-06-13
2,0ZN06O,Technology,50208,31013,110,2023-07-31
3,9BV15C,Music,360531,152565,202,2024-04-01
4,5SU87K,Travel,180381,146878,202,2023-07-03


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [5]:
num_interactions = 1_000_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

,interaction_id,user_id,video_id,interaction_type,watch_time
0,1353-32-0695,UC646D,3RE11Y,skip,20
1,1878-01-4191,DI752B,1GM21U,like,14
2,0786-41-2468,NB280G,1LA19B,skip,235
3,8740-17-7171,YY424S,7UV11T,skip,24
4,7741-58-6354,EX929F,8DC52V,skip,249


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [6]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 0 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 100}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

{"expectation_type": "expect_column_distinct_values_to_be_in_set", "kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "meta": {}}

In [7]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

{"expectation_type": "expect_column_values_to_be_dateutil_parseable", "kwargs": {"column": "upload_date"}, "meta": {}}

In [8]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "watch_time", "min_value": 0}, "meta": {}}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [9]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/17565
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [10]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

2024-10-01 13:31:21,761 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1239373
2024-10-01 13:31:23,525 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1239373


Uploading Dataframe: 0.00% |          | Rows 0/25000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [11]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

2024-10-01 13:31:34,511 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1237342
2024-10-01 13:31:36,108 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1237342


Uploading Dataframe: 0.00% |          | Rows 0/25000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: videos_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/videos_1_offline_fg_materialization/executions
Done ✅


In [12]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1238365
2024-10-01 13:31:50,058 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1238365


Uploading Dataframe: 0.00% |          | Rows 0/1000000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/interactions_1_offline_fg_materialization/executions
Done ✅


## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [13]:
video_interactions_df = pd.merge(
    data_interactions_df, 
    data_video_df, 
    on='video_id', 
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df, 
    data_users_df, 
    on='user_id', 
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'], 
    axis=1, 
    inplace=True,
)

ranking_df.head()

,user_id,video_id,category,views,likes,video_length,upload_date,gender,age,country,label
0,UC646D,3RE11Y,Comedy,24940,16465,203,2022-11-16,Female,77,Nepal,0
1,UC646D,6ZH27U,Entertainment,32439,31962,142,2023-08-10,Female,77,Nepal,0
2,UC646D,7WR26K,News,117476,12510,136,2024-03-28,Female,77,Nepal,0
3,UC646D,2RZ41M,Travel,235930,147583,33,2023-11-25,Female,77,Nepal,1
4,UC646D,6EN02I,Sports,265,128,222,2023-11-13,Female,77,Nepal,0


In [14]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

2024-10-01 13:33:15,634 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/1239374


Uploading Dataframe: 0.00% |          | Rows 0/1002532 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/ranking_1_offline_fg_materialization/executions
Done ✅


---